In [1]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
import getpass
import os
os.environ["OPENAI_API_KEY"] = "sk-"

In [2]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader


In [3]:
base_url = "http://127.0.0.1:9779/v1"
loader = TextLoader("./index.md")
docs = loader.load()


In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings
model_name = "/data/lixubin/models/m3e-base"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/data/lixubin/RetrieveQA/.env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048)
chunked_docs = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(
    chunked_docs,
    hf
)

In [19]:
chunked_docs[0]

Document(page_content="要乐奈（日语：要（かなめ） 楽奈（らーな））是由日本娱乐公司Bushiroad策划的次世代少女乐队企划《BanG Dream!》及其衍生作品的登场角色。代表色为#77DD77。\n\n简介\n\n动画立绘\nMyGO!!!!!的吉他手，花咲川女子学园初中三年级学生，在LiveHose「RiNG」里神出鬼没的古怪女孩，因为觉得乐队很有趣所以加入了乐队。\n\n喜欢的食物是抹茶和荞麦面，不喜欢韭菜和山药泥。\n\n爱好是和猫玩，本人也是相当的猫系少女，甚至生日也是“猫の日”2月22日。\n\n是左撇子。[2]\n\n喜欢穿宽松的衣服。\n\n是BanG Dream!企划第一位异色瞳角色。\n\n【自己紹介】楽奈。 [1/1]显示视频\n【一問一答】楽奈編 [1/1]显示视频\n经历\n\t以下内容含有剧透成分，可能影响观赏作品兴趣，请酌情阅读\n动画\n折叠\n要乐奈是原LIVE HOUSE SPACE店长都筑诗船的孙女，一开始活动于SPACE，在SPACE关门后失去了“容身之地”。在RiNG开业后时常提着外婆的吉他到这边来玩。\n\n在爱音、爽世和立希于RiNG咖啡厅内讨论成立乐队的事情时，自顾自地走上舞台开始吉他solo.\n\n灯，立希，爽世，爱音开始排练后，乐奈擅自闯进排练室，仅看了一眼《春日影》的乐谱就能演奏出来，给只会弹和弦的爱音带来了极大的打击。称灯为“有趣的女人”。\n\n立希作出新曲后，又擅自闯进排练室，演奏立希的新曲，刺激立希想要作出更好的曲子。结果立希因无法作出超越姐姐和祥子的乐曲而陷入了自我怀疑，担心的爱音和灯来到花咲川找到立希，偶遇了正在逗猫的乐奈，大家这才知道乐奈就读于花咲川女子学园初中三年级。\n\n因为好玩，乐奈决定加入这个四人乐队进行第一次演出。在Live上，乐奈在演奏完不太成功的碧天伴走后听到灯的MC重力小作文感到非常有趣，开始配乐并即兴演奏了之前看过一眼谱子的春日影，此举最终发展成了五人其实是四个人一起演奏/演唱春日影猫猫什么都不知道，猫猫只想爽弹。这直接导致了台下的丰川祥子情绪失控破防而跑出Live House；看见了此景的长崎爽世也跟着情绪失控了，Live成功后爽世在后台大发雷霆，此时乐奈已经离开了后台。\n\n爽世缺席练习后，因众人迷茫，乐奈称她们是“无趣的女人”，离开了乐队。在这

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="/data/lixubin/models/Qwen/Qwen1.5-32B-Chat" ,base_url=base_url)

In [7]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [8]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


In [9]:
type(prompt)
print(prompt)

input_variables=['context', 'question'] metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


In [10]:

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [11]:
type(rag_chain)

langchain_core.runnables.base.RunnableSequence

In [12]:
rag_chain.invoke("MyGO是什么?")

'MyGO是乐队的名字，由高松灯提议并加上五个感叹号确定为"MyGO!!!!!”。'

In [13]:
llm.invoke("做个自我介绍吧")

AIMessage(content='你好，我叫通义千问，是由阿里云开发的预训练语言模型。我被设计用来回答各种问题、提供信息和与用户进行对话。无论你对科技、文化、历史、数学、语言，还是其他任何主题感兴趣，我都会尽力为你提供准确和有用的答案。如果你有任何问题或需要帮助，欢迎随时向我提问。让我们开始交流吧！', response_metadata={'token_usage': {'completion_tokens': 83, 'prompt_tokens': 22, 'total_tokens': 105}, 'model_name': '/data/lixubin/models/Qwen/Qwen1.5-32B-Chat', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5538ab79-cb76-410b-90ff-ff8b0b1785c3-0')

In [14]:
# Get the prompt to use - you can modify this!
prompt_2 = hub.pull("hwchase17/self-ask-with-search")